# **Data Cleaning Project: Nashville Housing Dataset**

## **Load the Dataset**

### **Create a Copy of the Original Table**

In [1]:
-- Create a copy of the Housing_Data table
SELECT * 
INTO Housing_Data_Clean
FROM Housing_Data;

(56477 rows affected)

Total execution time: 00:00:00.126

**Comment:** This is to keep the original data intact and unaltered.

## **Data Type Verification**

### **Generate Data Types Report for All Columns**

In [2]:
-- Retrieve all columns and their corresponding data types
SELECT COLUMN_NAME as ColumnName, DATA_TYPE as DataType
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Housing_Data_Clean';

(19 rows affected)

Total execution time: 00:00:00.105

ColumnName,DataType
UniqueID,int
ParcelID,nvarchar
LandUse,nvarchar
PropertyAddress,nvarchar
SaleDate,date
SalePrice,money
LegalReference,nvarchar
SoldAsVacant,nvarchar
OwnerName,nvarchar
OwnerAddress,nvarchar


**Comment:** All columns are correctly assigned their respective data types.

## **Check for Missing Values**

### **Generate Null Count Report for All Columns**

In [3]:
-- Declare variables
DECLARE @TableName NVARCHAR(50)
SET @TableName = 'Housing_Data_Clean'

-- Initialize dynamic SQL string
DECLARE @SQL NVARCHAR(MAX) = ''

-- Construct SQL to count null values for each column
SELECT @SQL = @SQL + 
    'SELECT ''' + COLUMN_NAME + ''' AS column_name, 
    COUNT(CASE WHEN [' + COLUMN_NAME + '] IS NULL THEN 1 END) AS null_count
    FROM ' + @TableName + ' UNION ALL '
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = @TableName

-- Remove the last 'UNION ALL' from the SQL string
SET @SQL = LEFT(@SQL, LEN(@SQL) - 10)

-- Final query to retrieve results
SET @SQL = '
SELECT column_name, SUM(null_count) AS null_count
FROM (' + @SQL + ') AS Result
GROUP BY column_name
ORDER BY column_name'

-- Execute the dynamic SQL
EXEC sp_executesql @SQL;

Warning: Null value is eliminated by an aggregate or other SET operation.

(19 rows affected)

Total execution time: 00:00:00.485

column_name,null_count
Acreage,30462
Bedrooms,32320
BuildingValue,30462
FullBath,32202
HalfBath,32333
LandUse,0
LandValue,30462
LegalReference,0
OwnerAddress,30462
OwnerName,31216


**Comment:** Data exhibits varying degrees of missing values across its columns. Out of 19 columns, 12 contain a significant number of null values, ranging from 30,462 to 32,333. Only the PropertyAddress column has 29 missing values.

### **Further Investigation of Null Values**

In [ ]:
-- Retrieves all columns from the Housing_Data_Clean table
SELECT *
FROM Housing_Data_Clean;

**Comment:** Data are missing across majority of columns, which significantly compromise the completeness of each record. Out of 56477 rows, more than 30,000 rows in the dataset are affected similarly. A thorough examination of the dataset reveals that it is not possible to calculate the exact values of the missing fields where many columns data are missing in a single a row. Imputing such a large amount data would result in a susbtantial portion of information being estimated rather than observed. Hence, considering deletion of these rows may be reasonable. Columns  such as, YearBuilt, Bedrooms, FullBath, Halfbath have exhibit additional null values, particularly when BuildingValue is 0. ParcelID serving as an identifier, correlates closely with PropertyAddress. If we know the ParcelID, we can retrieve the missing PropertyAddress as long as another row with the same ParcelID and PropertyAddress exists. It also appears that Propertyaddress and OwnerAddress columns contain similar data, with the OwnerAddress column including additional state information.

## **Handle Missing Values**

### **Null Value Imputation**

In [4]:
-- Update table with missing PropertyAddresses
UPDATE Housing_Data_Clean
SET PropertyAddress = (
-- Search for PropertyAddress with null value and fill that by non-null PropertyAddress value from another record by matching their ParcelID
    SELECT TOP (1) hd.PropertyAddress
    FROM Housing_Data_Clean hd
    WHERE hd.ParcelID = Housing_Data_Clean.ParcelID
    AND hd.PropertyAddress IS NOT NULL
)
WHERE PropertyAddress IS NULL;

(29 rows affected)

Total execution time: 00:00:00.198

**Comment:** A total of 29 cells were imputed of the PropertyAddress column. Performing null value imputation before deleting rows ensures that efforsts were made to retain as much information as possible from the dataset.

### **Remove Rows with Many Null Values**

In [5]:
-- Delete rows where 11 column values are missing
DELETE 
FROM Housing_Data_Clean
WHERE
-- Check if column is null
Acreage IS NULL
AND Bedrooms IS NULL
AND FullBath IS NULL
AND HalfBath IS NULL
AND LandValue IS NULL
AND OwnerAddress IS NULL
AND OwnerName IS NULL
AND TaxDistrict IS NULL
AND TotalValue IS NULL
AND YearBuilt IS NULL;

(30462 rows affected)

Total execution time: 00:00:00.092

**Comment:** 30462 rows were deleted due to having null values across 11 columns. There are still null values present in the BuildingValue, YearBuilt, Bedrooms, FullBath, HalfBath and OwnerName columns. However, unlike the previously encountered random null values,  these null values may follow a discernible pattern or are associated with specific factors.

## **Check for Formatting Errors**

### **Check for Incorrect Date Format**

In [6]:
-- Convert SaleDate column values to date data type and filters out rows if conversion fails
SELECT *
FROM Housing_Data_Clean
WHERE TRY_CONVERT(date, SaleDate) IS NULL;

(0 rows affected)

Total execution time: 00:00:00.012

UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,OwnerName,OwnerAddress,Acreage,TaxDistrict,LandValue,BuildingValue,TotalValue,YearBuilt,Bedrooms,FullBath,HalfBath


**Comment:** Returned 0 row which indicates correct date formats.

### **Check for Non-Numeric Values in Numeric Columns**

In [ ]:
-- Filters out non-numeric rows of each numeric columns
SELECT *
FROM Housing_Data_Clean
WHERE ISNUMERIC(UniqueID) = 0 
OR ISNUMERIC(SalePrice) = 0
OR ISNUMERIC(Acreage) = 0
OR ISNUMERIC(LandValue) = 0
OR ISNUMERIC(BuildingValue) = 0
OR ISNUMERIC(TotalValue) = 0
OR ISNUMERIC(YearBuilt) = 0
OR ISNUMERIC(Bedrooms) = 0
OR ISNUMERIC(FullBath) = 0
OR ISNUMERIC(HalfBath) = 0;

**Comment:** Returned 2002 rows in total. However, These rows were filtered out due to the presence of nall values.

### **Check for Leading and Trailing Spaces**

In [8]:
-- Filters out rows with leading/trailing spaces
SELECT *
FROM Housing_Data_Clean
WHERE LTRIM(RTRIM(UniqueID)) <> UniqueID
OR LTRIM(RTRIM(ParcelID)) <> ParcelID
OR LTRIM(RTRIM(LandUse)) <> LandUse
OR LTRIM(RTRIM(PropertyAddress)) <> PropertyAddress
OR LTRIM(RTRIM(SaleDate)) <> SaleDate
OR LTRIM(RTRIM(SalePrice)) <> SalePrice
OR LTRIM(RTRIM(LegalReference)) <> LegalReference
OR LTRIM(RTRIM(SoldAsVacant)) <> SoldAsVacant
OR LTRIM(RTRIM(OwnerName)) <> OwnerName
OR LTRIM(RTRIM(OwnerAddress)) <> OwnerAddress
OR LTRIM(RTRIM(Acreage)) <> Acreage
OR LTRIM(RTRIM(TaxDistrict)) <> TaxDistrict
OR LTRIM(RTRIM(LandValue)) <> LandValue
OR LTRIM(RTRIM(BuildingValue)) <> BuildingValue
OR LTRIM(RTRIM(TotalValue)) <> TotalValue
OR LTRIM(RTRIM(YearBuilt)) <> YearBuilt
OR LTRIM(RTRIM(Bedrooms)) <> Bedrooms
OR LTRIM(RTRIM(FullBath)) <> FullBath
OR LTRIM(RTRIM(HalfBath)) <> HalfBath;

(1 row affected)

Total execution time: 00:00:00.239

UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,OwnerName,OwnerAddress,Acreage,TaxDistrict,LandValue,BuildingValue,TotalValue,YearBuilt,Bedrooms,FullBath,HalfBath
9095,063 12 0 063.00,VACANT RES LAND,"HADLEYS BEND BLVD, OLD HICKORY",2013-10-31,19000.00,20131101-0113875,Yes,"DANNER-ELLER GOLF PROPERTIES, INC.","0 HADLEYS BEND BLVD, OLD HICKORY, TN",1.15,GENERAL SERVICES DISTRICT,17600.00,0.00,17600.00,NULL,NULL,0,0


**Comment:** Only one row of PropertyAddress column has leading/trailing related issue.

### **Manually Check for Other Formatting Errors**

In [ ]:
-- View all columns
SELECT *
FROM Housing_Data_Clean;

**Comment:** LandUsePropertyaddress, LegalReference, OwnerName, and OwnerAddress Columns have mixed spacing inside texts. There are typing issues present in columns LandUse (e.g. mix of VACANT RES LAND and VACANT RESIDENTIAL LAND), Legalreference (e.g. shorter or longer cell values than expected), SoldAsVacant (e.g. mix of N and No), OwnerName (mix of comma and no comma).

## **Handle Formatting Errors**

### **Remove Leading and Trailing Spaces**

In [9]:
-- Trim and update cell with leading/trailing spaces
UPDATE Housing_Data_Clean
SET PropertyAddress = LTRIM(RTRIM(PropertyAddress))
WHERE UniqueID = 9095;

(1 row affected)

Total execution time: 00:00:00.029

**Comment:** PropertyAddress value for row with UniqueID = 9095 was updated to remove any leading/trailing spaces.

### **Standardize Spaces**

In [10]:
-- Standardize spaces in the PropertyAddress column by replacing multiple spaces with a single space
UPDATE Housing_Data_Clean
SET PropertyAddress = REPLACE(PropertyAddress, '  ', ' ');

-- Standardize spaces in the OwnerName column by replacing multiple spaces with a single space
UPDATE Housing_Data_Clean
SET OwnerName = REPLACE(OwnerName, '  ', ' ');

-- Standardize spaces in the OwnerAddress column by replacing multiple spaces with a single space
UPDATE Housing_Data_Clean
SET OwnerAddress = REPLACE(OwnerAddress, '  ', ' ');

(26015 rows affected)

(26015 rows affected)

(26015 rows affected)

Total execution time: 00:00:00.641

**Comment:** In this way spacing between words in the PropertyAddress, OwnerName, and OwnerAddress columns are standardized to 1.

### **Resolving Typing Errors**

In [11]:
-- Update the LandUse column to replace 'GREENBELT/RES GRRENBELT/RES' with 'GREENBELT/RES'
UPDATE Housing_Data_Clean
SET LandUse = 'GREENBELT/RES'
WHERE LandUse = 'GREENBELT/RES
GRRENBELT/RES';

-- Update the LandUse column to replace 'VACANT RES LAND' with 'VACANT RESIDENTIAL LAND'
UPDATE Housing_Data_Clean
SET LandUse = 'VACANT RESIDENTIAL LAND'
WHERE LandUse = 'VACANT RES LAND';

-- Update the LegalReference column where '10160622-0063516' is corrected to '20160622-0063516' 
UPDATE Housing_Data_Clean
SET LegalReference = '20160622-0063516'
WHERE LegalReference = '10160622-0063516';

-- Update the LegalReference column where '21040903-0080324' is corrected to '20140903-0080324' 
UPDATE Housing_Data_Clean
SET LegalReference = '20140903-0080324'
WHERE LegalReference = '21040903-0080324';

-- Update the LegalReference column where '20150310 -0020554' is corrected to '20150310-0020554'
UPDATE Housing_Data_Clean
SET LegalReference = '20150310-0020554'
WHERE LegalReference = '20150310 -0020554';

-- Update the LegalReference column where '20105031- 002049' is corrected to '20150310-0020497' 
UPDATE Housing_Data_Clean
SET LegalReference = '20150310-0020497'
WHERE LegalReference = '20105031- 002049';

-- Update the LegalReference column where '25015081- 008375' is corrected to '20150819-0083759' 
UPDATE Housing_Data_Clean
SET LegalReference = '20150815-0008375'
WHERE LegalReference = '25015081- 008375';

-- Update the LegalReference column where '20160408-00339999' is corrected to '20160408-0033999' 
UPDATE Housing_Data_Clean
SET LegalReference = '20160408-0033999'
WHERE LegalReference = '20160408-00339999';

-- Update the LegalReference column where '20130610-00588852' is corrected to '20130610-0058852'
UPDATE Housing_Data_Clean
SET LegalReference = '20130610-0058852'
WHERE LegalReference = '20130610-00588852';

-- Update the LegalReference column where '20015100- 010146' is corrected to null 
UPDATE Housing_Data_Clean
SET LegalReference = NULL
WHERE LegalReference = '20015100- 010146';

-- Update the LegalReference column where '-2020988' is corrected to null 
UPDATE Housing_Data_Clean
SET LegalReference = NULL
WHERE LegalReference = '-2020988';

-- Update the LegalReference column where '-2020879' is corrected to null 
UPDATE Housing_Data_Clean
SET LegalReference = NULL
WHERE LegalReference = '-2020879';

-- Update the LegalReference column where '-2016598' is corrected to null 
UPDATE Housing_Data_Clean
SET LegalReference = NULL
WHERE LegalReference = '-2016598';

-- Update the SoldAsVacant column where 'No' and 'N' is corrected to 'NO'
UPDATE Housing_Data_Clean
SET SoldAsVacant = 'NO'
WHERE SoldAsVacant = 'No' or SoldAsVacant = 'N';

-- Update the SoldAsVacant column where 'Yes' and 'Y' is corrected to 'YES'
UPDATE Housing_Data_Clean
SET SoldAsVacant = 'YES'
WHERE SoldAsVacant = 'Yes' or SoldAsVacant = 'Y';

-- Add comma before 'LLC' if it is not already present in the OwnerName column
UPDATE Housing_Data_Clean
SET OwnerName = REPLACE(OwnerName, ' LLC', ', LLC')
WHERE OwnerName LIKE '% LLC' AND OwnerName NOT LIKE '%, LLC';

(3 rows affected)

(445 rows affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(24784 rows affected)

(1231 rows affected)

(117 rows affected)

Total execution time: 00:00:00.564

**Comment:** Typing errors were resolved by replacing the wrong one with the actual value where possible. If the actual value could not be understood from the available data, the cell value was set to null.

## **Check for Duplicate Rows**

In [12]:
-- Use a CTE to assign row numbers to rows partitioned by ParcelID, SaleDate, SalePrice, and LegalReference. with an arbitrary ordering
WITH CTE AS (
    SELECT ParcelID, SaleDate, SalePrice, LegalReference,
        ROW_NUMBER() OVER(PARTITION BY ParcelID, SaleDate, SalePrice, LegalReference ORDER BY(SELECT NULL)) AS RowNum
    FROM Housing_Data_Clean
)
-- Retrieve rows from the CTE where the row number is greater than 1
SELECT *
FROM CTE
WHERE RowNum > 1;

(48 rows affected)

Total execution time: 00:00:00.087

ParcelID,SaleDate,SalePrice,LegalReference,RowNum
050 02 0 020.00,2015-03-06,65000.00,20150310-0020497,2
081 02 0 144.00,2015-02-02,57000.00,20150205-0010843,2
081 07 0 265.00,2015-02-17,65000.00,20150223-0015122,2
081 10 0 313.00,2015-02-20,35000.00,20150224-0015904,2
081 11 0 168.00,2015-02-13,44500.00,20150218-0013602,2
081 11 0 495.00,2015-02-09,36500.00,20150210-0012450,2
081 15 0 263.00,2015-02-12,55000.00,20150218-0013742,2
081 15 0 472.00,2015-02-20,35000.00,20150223-0015257,2
083 14 0 153.00,2013-06-07,449830.00,20130610-0058852,2
090 08 0 191.00,2015-02-13,169000.00,20150219-0014430,2


**Comment:** At this point, 48 duplicate rows are present in the dataset.

## **Remove Duplicate Rows**

In [13]:
-- Use a CTE to assign row numbers to rows partitioned by ParcelID, SaleDate, SalePrice, and LegalReference. with an arbitrary ordering
WITH CTE AS (
    SELECT ParcelID, SaleDate, SalePrice, LegalReference,
        ROW_NUMBER() OVER(PARTITION BY ParcelID, SaleDate, SalePrice, LegalReference ORDER BY(SELECT NULL)) AS RowNum
    FROM Housing_Data_Clean
)
-- Delete rows from the CTE where the row number is greater than 1
DELETE FROM CTE
WHERE RowNum > 1;

(48 rows affected)

Total execution time: 00:00:00.051

**Comment:** After removing 48 duplicates, now retaining only one copy of each unique combination ParcelID, SaleDate, SalePrice, and LegalReference.

## **Identify Outliers**

In [ ]:
-- Calculate quartiles
WITH Quartiles AS (
    SELECT 
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY SalePrice) OVER(PARTITION BY 1) AS Q1,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY SalePrice) OVER(PARTITION BY 1) AS Q3
    FROM Housing_Data_Clean
),
-- Calculate IQR
IQR AS (
    SELECT 
        MAX(Q1) AS Q1,
        MAX(Q3) AS Q3,
        MAX(Q3) - MAX(Q1) AS IQR
    FROM Quartiles
)
-- Identify outliers
SELECT *
FROM Housing_Data_Clean
WHERE SalePrice < (SELECT Q1 - 1.5 * IQR FROM IQR) OR SalePrice > (SELECT Q3 + 1.5 * IQR FROM IQR);

**Comment:** 7.24% of the SalePrice rows are outliers. However, without further investigation it is not possible to determine whether they are valid data points or errors.

## **Split Columns**

### **First Add New Columns to the Table**

In [14]:
-- Add new columns to the table
ALTER TABLE Housing_Data_Clean
ADD ProAddress NVARCHAR (50),
    ProCity NVARCHAR (50),
    OwnAddress NVARCHAR (50),
    OwnCity NVARCHAR (50),
    OwnState NVARCHAR (50);

Commands completed successfully.

Total execution time: 00:00:00.005

**Comment:** ProAddress and ProCity columns are to store extracted data from PropertyAddress. OwnAddress, OwnCity, and OwnState columns are to store extracted data from OwnerAddress column.

### **Populate New Columns**

In [15]:
-- Update the new columns by adding extracted data from PropertyAddrsess 
UPDATE Housing_Data_Clean
SET ProAddress = SUBSTRING(PropertyAddress, 1, CHARINDEX(',', PropertyAddress) - 1),
    ProCity = SUBSTRING(PropertyAddress, CHARINDEX(',', PropertyAddress) + 2, LEN(PropertyAddress)),

-- Update the new columns by adding extracted data from OwnerAddrsess 
    OwnAddress = PARSENAME(REPLACE(OwnerAddress, ',', '.'), 3),
    OwnCity = PARSENAME(REPLACE(OwnerAddress, ', ', '.'), 2),
    OwnState = PARSENAME(REPLACE(OwnerAddress, ', ', '.'), 1);

(25967 rows affected)

Total execution time: 00:00:00.743

**Comment:** Afer splitting PropertyAddress and OwnerAddress columns we now have the separate columns for the city and state data, which can be benificial for analysis and reporting purposes, as it allows easier querying and filtering based on geographic criteria.

## **Remove Unused Columns**

In [16]:
-- Remove specified columns from the table
ALTER TABLE Housing_Data_Clean
DROP COLUMN PropertyAddress, 
            OwnerAddress;

Commands completed successfully.

Total execution time: 00:00:00.004

**Comment:** Original PropertyAddress and OwnerAddress columns were deleted to avoid redundancy, as new columns are already in place holding the same data.